# This is a notebook for creating a webscraper for house prices and features.

## Wel'll scrape the data, wrangle it, and then insert it intoa database for later use.


In [1]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

import re
import string
import json
import unicodedata

import sqlite3
from sqlite3 import Error

In [2]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
    return conn

def create_house(conn, house_features):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO house_features(HOUSE_TYPE,
                                        HOUSE_ROOMS,
                                        ADDRESS,
                                        AREA,
                                        CITY,
                                        PRICE,
                                        SIZE,
                                        YEAR)
              VALUES(?,?,?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, house_features)
    conn.commit()
    return cur.lastrowid
    
def insert_data(conn,HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR):
    with conn:
        house_1 = (HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR)
        create_house(conn, house_1)
 

In [37]:
# conn = create_connection(r"../Data/house.db")
url = 'https://www.etuovi.com/'
wd = webdriver.Chrome(executable_path=r'../Data/chromedriver.exe')
wd.get(url)
time.sleep(5)
# Accept cookies
try:
    wd.find_element_by_xpath('/html/body/div[4]/div/div/div[2]/div[3]/button[2]').click()
    time.sleep(2)
except:
    pass
# Search for city 'Helsinki'
time.sleep(5)
wd.find_element_by_id('location').send_keys("Helsinki")
time.sleep(2)
# Click the search button
wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div/div[1]/div/div[2]/div/form/div/div[2]/div[2]/button').click()
time.sleep(1)
wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div/div[1]/div/div[2]/div/form/div/div[2]/div[2]/button').click()
time.sleep(5)

max_n = int(unicodedata.normalize("NFKD", wd.find_elements_by_id('searchResultCount')[0].get_attribute('innerText')).replace(" ", ""))

wd.find_elements_by_id('searchResultCount')[0].get_attribute('innerText')
t = wd.find_element_by_id('announcement-list')
print(t.get_attribute('innerText').splitlines()[1:50])
time.sleep(3)
wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
time.sleep(3)
wd.find_element_by_xpath('/html/body/div[2]/div/div[3]/div/div[2]/div[2]/div[1]/div/div[5]/div[1]/div[7]/button').click()
time.sleep(5)
wd.find_elements_by_id('searchResultCount')[0].get_attribute('innerText')
t2 = wd.find_element_by_id('announcement-list')
print(t2.get_attribute('innerText').splitlines()[1:50])


['UUTTA!', '', 'Jatkossa näet hakutuloslistalla asuinpinta-alan lisäksi myös asunnon kokonaispinta-alan. Jos pinta-alat ovat samat, listalla näytetään vain toinen.', '', 'SELVÄ', 'UUSI', '24H', 'Kerrostalo | 2H + KK + S', 'Kallvikintie 59 B 20, Vuosaari, Helsinki', 'HINTA', '19\xa0355,55\xa0€', 'KOKO', '52,5 m²', 'VUOSI', '2000', 'UUSI', '24H', 'TI 21.12.', 'Klo 00:00', 'Kerrostalo | 3h + k + kph + wc (yj. mukaan 3 h + k + kph + wc)', 'Klaarantie 1, Lauttasaari, Helsinki', 'HINTA', '498\xa0000\xa0€', 'KOKO', '73 m²', 'VUOSI', '1967', 'UUSI', '24H', 'Kerrostalo | 2h, k, vh, kph, sauna, parvi', 'Kruunuvuorenkatu 5, Katajanokka, Helsinki', 'HINTA', '464\xa0000\xa0€', 'KOKO', '47 / 54 m²', 'VUOSI', '1897', 'UUSI', '24H', 'ondemand_video', 'Kerrostalo | 3h + k + kph + lasitettu p (yhtiöjärjestyksen mukaan 3h + k)', 'Raaseporintie 1 C, Puotinharju, Helsinki', 'HINTA', '215\xa0000\xa0€', 'KOKO', '68 m²', 'VUOSI', '1962', 'UUSI']
['UUTTA!', '', 'Jatkossa näet hakutuloslistalla asuinpinta-alan 

# TODO:
 - Everything is now working, need to create easily useable functions for each segment
    1. Get the correct page and search def x(city,url):
    2. Read page contents def y(wd): -> List
    3. Insert first page contents into DB
    4. Go to next page def z(wd)
    5. -> Back to step 2.
- Add error handling
    - Scraping
    - DB insert
- Move whole project into a .py file

In [172]:
i=0
while i < len(t) :
    line = t[i] # Can't use enumerate because we want to be able to jump forward in the list
    if '|' in line:
        HOUSE_TYPE = re.match('.+?(?=\|)',line).group().strip()
        # print(HOUSE_TYPE)
        HOUSE_ROOMS = re.search("(?<=\|).*?(?=(?:\()|$)" ,line).group()
        # print(HOUSE_ROOMS)
        # Location
        ADDRESS,AREA,CITY = [x.strip() for x in t[i+1].split(",")]
        # print(ADDRESS)
        # Price (rounding to nearest int)
        try:
            PRICE = int(float(unicodedata.normalize("NFKD",t[i+3]).replace('€',"").replace(" ","").replace(',','.')))
        except:
            PRICE = 0
        # print(PRICE)
        # Size (rounding to nearest int)
        try:
            SIZE = int(float(unicodedata.normalize("NFKD",t[i+5].split(" ")[0]).replace(',','.')))
        except:
            SIZE = 0
        # print(SIZE)
        # Year
        try:
            YEAR = int(t[i+7].strip())
        except:
            YEAR = 0
        # print(YEAR)
        # insert_data(conn,HOUSE_TYPE,HOUSE_ROOMS,ADDRESS,AREA,CITY,PRICE,SIZE,YEAR)
    i+=1

Kerrostalo
 3h + k + kph + lasitettu p 
Raaseporintie 1 C
215000
68
1962
Kerrostalo
 3 h, k, kph, wc, piha
Radiokatu 14
348000
77
1987
Kerrostalo
 2h + k + kh, jossa WC
Kolmas linja 5
250000
50
1940
Kerrostalo
 2h + k + kh + patio
Ruoholahdenkatu 10 B
415000
55
1940
Omakotitalo
 5-6h, keittiö, saunaosasto, kylpyhuone, wc, khh
Pohjanpellontie 9
428000
160
1955
Kerrostalo
 3h + k + kph + s + wc
Pietarinkatu 10
1098000
96
1906
Kerrostalo
 2h + kk + s + lasp
Malmin raitti 28
201000
59
1984
Kerrostalo
 1h + kk + kh 
Kanneltie 2
158000
30
1960
Kerrostalo
 1h + kk + kh 
Fredrikinkatu 31
245000
20
1900
Kerrostalo
 2H + KK + S
Pohjavedenportti 2
15494
45
1999
Kerrostalo
 3H + K + S
Laivalahdenkaari 15
21543
65
1997
Kerrostalo
 2h, k, wc / kph, eteinen, p 
Laajalahdentie 28
399000
58
1956
Kerrostalo
 2h + avok + kph + parveke 
Hiihtomäentie 18 D
268000
47
1956
Kerrostalo
 2h, k, kph, parveke
Pajalahdentie 4
429000
51
1939
Kerrostalo
 1h, kk, kph, vh, lasitettu parveke
Kauppalantie 16
216000
30
1

In [159]:
# Used to create the table from scratch (debugging mostly)
# conn = create_connection(r"../Data/house.db")   
# cursor = conn.cursor()

# cursor.execute("""CREATE TABLE IF NOT EXISTS house_features (
#     ID integer PRIMARY KEY,
#     HOUSE_TYPE text,
#     HOUSE_ROOMS text,
#     ADDRESS text,
#     AREA text,
#     CITY text,
#     PRICE integer,
#     SIZE integer,
#     YEAR integer
# );
# """)
# conn.commit()
# conn.close()

In [168]:
# def delete_all_houses(conn):
#     """
#     Delete all rows in the tasks table
#     :param conn: Connection to the SQLite database
#     :return:
#     """
#     sql = 'DELETE FROM house_features'
#     cur = conn.cursor()
#     cur.execute(sql)
#     conn.commit()
# delete_all_houses(conn)